<a href="https://colab.research.google.com/github/1hamzaiqbal/MFCLIP_acv/blob/hamza%2Fdiscrim/tsne_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# t-SNE visualization for CLIP features

This notebook:
1. Sets up the repo and environment (Colab-ready)
2. Downloads Oxford Pets dataset
3. Extracts CLIP image features and labels
4. Runs t-SNE visualization to show discrimination ability


In [6]:
# 0) GPU + repo setup
!nvidia-smi
%cd /content
!git clone -b hamza/discrim https://github.com/1hamzaiqbal/MFCLIP_acv
%cd MFCLIP_acv



Tue Nov  4 02:32:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   43C    P8             17W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
# 1) Install dependencies
!pip install torch torchvision timm einops yacs tqdm opencv-python scikit-learn scipy pyyaml ruamel.yaml pytorch-ignite foolbox pandas matplotlib seaborn wilds ftfy



In [8]:
# 2) Download Oxford Pets dataset
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from pathlib import Path

root = Path("/content/data/oxford_pets")
root.mkdir(parents=True, exist_ok=True)
_ = OxfordIIITPet(root=str(root), download=True, transform=transforms.ToTensor())
print("Oxford Pets downloaded to", root)

# Also fetch annotations
%cd /content
!mkdir -p /content/data/oxford_pets
!wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xf images.tar.gz -C /content/data/oxford_pets
!tar -xf annotations.tar.gz -C /content/data/oxford_pets
!ls /content/data/oxford_pets/annotations | head



Oxford Pets downloaded to /content/data/oxford_pets
/content
list.txt
README
test.txt
trainval.txt
trimaps
xmls


In [1]:
# 3) Mount Google Drive for checkpoints

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# 4) Configure checkpoints for fine-tuned heads

import os

arcfacesigmoid_weights = '/content/drive/MyDrive/grad/comp_vision/hanson_loss/oxford_pets/RN50_ArcFaceSigmoid_300ep.pth'
siglip_weights = '/content/drive/MyDrive/grad/comp_vision/hanson_loss/oxford_pets/RN50_SigLipHead_300ep.pth'

available_checkpoints = {
    'ArcFaceSigmoid': arcfacesigmoid_weights,
    'SigLipHead': siglip_weights,
}

for name, ckpt in available_checkpoints.items():
    if ckpt and os.path.isfile(ckpt):
        print('{}: found checkpoint at {}'.format(name, ckpt))
    else:
        print('{}: checkpoint not found at {}; this head will be skipped.'.format(name, ckpt))


In [ ]:
# 5) Run t-SNE comparisons across heads

import os
from argparse import Namespace

from IPython.display import Image, display

from scripts import plot_tsne

split = 'test'  # Choose among 'train', 'val', or 'test'
samples_per_class = 20  # Limit per-class samples for faster t-SNE
output_path = f'/content/MFCLIP_acv/tsne_{split}_comparison.png'

arcface_ckpt = arcfacesigmoid_weights if arcfacesigmoid_weights and os.path.isfile(arcfacesigmoid_weights) else None
siglip_ckpt = siglip_weights if siglip_weights and os.path.isfile(siglip_weights) else None

args = Namespace(
    root='/content/data',
    dataset_config_file='configs/datasets/oxford_pets.yaml',
    config_file='configs/trainers/CoOp/rn50.yaml',
    split=split,
    device='cuda',
    batch_size=256,
    max_samples=None,
    samples_per_class=samples_per_class,
    class_filter=None,
    include_vanilla=True,
    arcface_checkpoint=arcface_ckpt,
    siglip_checkpoint=siglip_ckpt,
    feature_stage='backbone',
    perplexity=30.0,
    random_state=0,
    output=output_path,
    cmap='tab20',
    point_size=12.0,
    alpha=0.7,
    show_legend=True,
    dpi=200,
    trainer='ZeroshotCLIP',
    backbone='RN50',
    opts=[],
)

print('Running t-SNE with the following checkpoints:')
print('  - Vanilla CLIP (always included)')
print('  - ArcFaceSigmoid: {}'.format(arcface_ckpt if arcface_ckpt else 'skipped'))
print('  - SigLipHead: {}'.format(siglip_ckpt if siglip_ckpt else 'skipped'))

plot_tsne.main(args)

if os.path.isfile(output_path):
    display(Image(filename=output_path))
else:
    raise FileNotFoundError('Expected t-SNE image was not created at {}'.format(output_path))
